In [5]:
# Activate project
using Pkg
Pkg.activate("/Users/alessiogiorlandino/Desktop/SAM/Project.toml")
Base.active_project()
# Resolve dependencies
Pkg.resolve()

  Activating project at `~/Desktop/SAM`


  No Changes to `~/Desktop/SAM/Project.toml`
  No Changes to `~/Desktop/SAM/Manifest.toml`


In [6]:
import Rfam, Infernal, Logomaker, FASTX
using Optimisers: Adam
using EllipsisNotation: (..)
using FillArrays: Falses
import Makie, CairoMakie
using Makie: @colorant_str
import SamApp
using LinearAlgebra
using Statistics

using CSV
using DataFrames

using BioSequences: LongRNA, @rna_str
using Statistics: mean, std, cor

In [7]:
MSA_df=SamApp.rf00162_hits_taxonomy()


Row,id,sequence,organism,taxonomy
,String31,String,String,String
1,CM000733.1/548881-548775,CUCUUAUCAAGAGAGGUGGAGGGAUCGGCCCUAUGAAACCUCGGCAACCCCUAUUUUAUUUAGGAAGGUGCCAAUUCCUUUAGAAUUU-AAAUUCUAAAAGAUAAGAA,Bacillus cereus Rock3-44,Bacteria; Bacillota; Bacilli; Bacillales; Bacillaceae; Bacillus; Bacillus cereus group
2,MLYR01000084.1/7469-7577,UUCUUAUCCAGAGAGGUGGAGGGACUGGCCCUGCGAAACCUCAGCAACCCCUGAUGAUUUCAGGAAGGUGCUAAUUCCAACAGAGUUGACAACUCUGGGAGAUAAGAG,Bacillus sp. MUM 116,Bacteria; Bacillota; Bacilli; Bacillales; Bacillaceae; Bacillus
3,AZTB01000003.1/66821-66716,CUCUUAUCUAGAGAGGUGGAGGGACUGGCCCUAUGAAACC-CGGCAACCUGUGUUUUAAACACAA-GGUGCCAAUUCCUGCGGUAUAAGAUAUACCGAGAGAUGAGAG,Caloranaerobacter azorensis H53214,Bacteria; Bacillota; Tissierellia; Tissierellales; Thermohalobacteraceae; Caloranaerobacter
4,LOED01000010.1/15598-15489,UUCUUAUCAAGAGCGGUGGAGGGACUGGCCCAAUGAAACC-CGGCAACCCGCAUCGUAGAUGCGAAGGUGCCAAUUCCUGCAGGACGAAAUGUCCUGAGAGAUAAGAA,Fervidicola ferrireducens,Bacteria; Bacillota; Clostridia; Thermosediminibacterales; Thermosediminibacteraceae; Fervidicola
5,CP007739.1/1700701-1700817,UUCUUAUCAAGAGAGGUCGAGGGAAUGGCCCAAUGAAGCCUCAGCAACCUUCAAUGUCAUUGAAAAGGUGCUAAUUCCAGCAAGUUCAUUUAACUUGAAAGAUAAGAA,Bacillus methanolicus MGA3,Bacteria; Bacillota; Bacilli; Bacillales; Bacillaceae; Bacillus
6,CP020772.1/4059248-4059357,UUCUUAUCAAGAGAGGCAGAGGGACUGGCCCUGUGAAGCCUCAGCAACCACUGGUAGAUCCAGUAAGGUGCUACGUCCAGCAAGUUGUGACAACUUGGAAGAUAAGAA,Halobacillus mangrovi,Bacteria; Bacillota; Bacilli; Bacillales; Bacillaceae; Halobacillus
7,JRNX01000191.1/6612-6721,UUCUUAUCAAGAGAGGCAGAGGGACUGGCCCUGUGAAGCCUCAGCAACCACUGGUAUAUCCAGUAAGGUGCUACGUCCAGCAAGUUGUAACAACUUGGAAGAUGAGAA,Halobacillus sp. BBL2006,Bacteria; Bacillota; Bacilli; Bacillales; Bacillaceae; Halobacillus
8,LFXJ01000010.1/73373-73267,CUCUUAUCAAGAGUAGCUGAGGGACUGGCCCUAUGAAGCU-CGGCAACCUCCUUUUUUUUAAGGAAGGUGCUAAAUCCUGCAAACGAUAAUCGUUUGAAAGAUAAGAG,Lysinibacillus xylanilyticus,Bacteria; Bacillota; Bacilli; Bacillales; Bacillaceae; Lysinibacillus
9,KI629782.1/602788-602686,CUCUUAUCCAGAGAGGCAGAGGGACUGGCCCGAUGAAGCCUCGGCAACCGGAAUAA--A-AUUC--GGUGCCAAUUCCAGCAAAGCGCUUUGCUUUGAAAGAUAAGAG,Brevibacillus panacihumi W25,Bacteria; Bacillota; Bacilli; Bacillales; Paenibacillaceae; Brevibacillus


In [16]:
MSA_df=SamApp.rf00162_hits_taxonomy()
sp_taxonomy = split.(MSA_df[:,"taxonomy"], "; ")
phyla = fill("other", length(sp_taxonomy))
domain = fill("Bacteria", length(sp_taxonomy))

for i in 1:length(sp_taxonomy)
    if length(sp_taxonomy[i]) >= 2
        phyla[i] = sp_taxonomy[i][2]
        domain[i] = sp_taxonomy[i][1]
    end
end

MSA_df[!,:domain] = domain
MSA_df[!,:phylum] = phyla

domain_counts = sort!(combine(groupby(MSA_df, :domain), :domain => length => :count), :count, rev=true)


Row,domain,count
,String,Int64
1,Bacteria,6085
2,Eukaryota,21
3,unclassified sequences,6


In [17]:
phyla_counts = sort!(combine(groupby(MSA_df, :phylum), :phylum => length => :count), :count, rev=true)



Row,phylum,count
,String,Int64
1,Bacillota,3679
2,Bacteroidota,854
3,Actinomycetota,691
4,Pseudomonadota,187
5,Chloroflexi,142
6,Firmicutes,84
7,Fusobacteria,43
8,Deinococcus-Thermus,38
9,other,33


In [18]:
for j in 1:nrow(MSA_df)
    if counts.count[findall(counts.phylum.== MSA_df[j, :phylum])][1] < 200 
        MSA_df[!, :phylum][j] = "other"
    end
end

label_file = open("labels.txt","w+")
#write(label_file, "#$(counts.phylum)\n")
#labels: ["Bacillota", "Bacteroidota", "Actinomycetota", "other", "Pseudomonadota", "Chloroflexi"]
for label in MSA_df.phylum
    println(label_file, label)
end
close(label_file)